In [2]:
import django
import sys, os
import platform
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta
import numpy as np

if platform.node() == "srv-mcc-apsis":
    #sys.path.append("/home/galm/software/django/tmv/BasicBrowser/")
    sys.path.append('/home/leey/tmv/BasicBrowser/')
else:
    # local paths
    sys.path.append('/home/leey/Documents/Data/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

#import from appended path
import scoping.models as sm
import twitter.models as tm
import parliament.models as pm

from django.db.models import Sum, Count
from django.db.models.fields import DateField
from django.db.models.functions import Cast
import textwrap as tw
import datetime
import random

import igraph
#import mpld3

from adjustText import adjust_text

In [3]:
p = sm.Project.objects.get(pk=196)

In [4]:
end_date = datetime.datetime.strptime('2019-09-30-+0000', '%Y-%m-%d-%z')

In [5]:
# Get tweets related to the project
searches = tm.TwitterSearch.objects.filter(project=p)

ttweets = set(tm.Status.objects.filter(
    searches__in=searches,
    text__isnull=False,
    retweeted_status__isnull=True,
    created_at__lte=end_date
).values_list('id',flat=True))

In [6]:
print(len(ttweets))

117266


In [39]:
import pickle

pickle_out = open("ttweets.pickle","wb")
pickle.dump(ttweets, pickle_out)
pickle_out.close()

---

## Language Detection with TextBlob

In [7]:
import random
random.seed(123)

In [8]:
test_texts = tm.Status.objects.filter(
    id__in=random.sample(ttweets, 1000)
).values_list('text',flat=True)

In [26]:
df_test = pd.DataFrame({"text":list(test_texts[0:10])})

In [17]:
from textblob import TextBlob

In [34]:
language = []
for i in df_test.index:
    entry = df_test.loc[i]
    blob = TextBlob(entry["text"])
    language.append(blob.detect_language())
    
language

['de', 'de', 'de', 'de', 'en', 'en', 'es', 'de', 'de', 'de']

In [35]:
df_test["language"] = language

In [36]:
df_test

,text,language
0,Wir stehen hinter jeglichem Friedlichen Protes...,de
1,"#HambacherForst #HambiBleibt ""Als erstes Geric...",de
2,Jetzt erst recht: #Hambibleibt. Wünsche den Be...,de
3,Wir müssen den herrschenden zeigen das wir unb...,de
4,#Hambibleibt https://t.co/xESw54Mh90,en
5,10:07am Uprooting has started at #Hambacher fo...,en
6,@Oaktown1312 informa en directo de la situació...,es
7,"„Wenn Unrecht zu Recht gemacht wird, dann wird...",de
8,Gegen Naturzerstörung für die fossile Gewinnma...,de
9,Klarer Fall von Selbstschutz wie wir es von de...,de


---

## Language Detection with Google Cloud API

In [44]:
from google.cloud import translate


def sample_detect_language(project_id="YOUR_PROJECT_ID"):
    """Detecting the language of a text string."""

    client = translate.TranslationServiceClient()

    parent = client.location_path(project_id, "global")

    # Detail on supported types can be found here:
    # https://cloud.google.com/translate/docs/supported-formats
    response = client.detect_language(
        content="Hello, world!",
        parent=parent,
        mime_type="text/plain",  # mime types: text/plain, text/html
    )

    # Display list of detected languages sorted by detection confidence.
    # The most probable language is first.
    for language in response.languages:
        # The language detected
        print(u"Language code: {}".format(language.language_code))
        # Confidence of detection result for this language
        print(u"Confidence: {}".format(language.confidence))


ModuleNotFoundError: No module named 'google.cloud'

---

## Language from BaseModel

In [15]:
random.seed(123)
test_texts2 = tm.Status.objects.filter(
    id__in=random.sample(ttweets, 1000)
).values_list('text', 'lang')

In [16]:
df_test2 = pd.DataFrame(list(test_texts2), columns=['text', 'lang'])

In [66]:
#df_test2.to_csv("df_test.csv")

In [17]:
df_test2

,text,lang
0,Thanks @guardianeco for this wonderful article...,en
1,"#hambibleibt ""Steinewürfe auf PolizistInnen"", ...",None
2,"An alle Menschen die gerade in Köln sind, im P...",None
3,Wir sind raus für Heute. Ob es morgen eine Mah...,None
4,"Kampf um den Hambacher Forst: ""Mir tut der Ged...",None
5,Haben unsere Tweets bezüglich einer Räumung de...,None
6,Hambacher Forst bleibt. #MakeMeHappyIn3Words @...,None
7,Klettern für #HambiBleibt = Kindheitserinneru...,de
8,Am 8.12. ist wieder offenes Hambi-Vernetzungst...,de
9,Hambacher Wald retten! Heute winterlicher Wald...,None


In [20]:
is_de = df_test2["lang"]=="de"
df_de = df_test2[is_de]

In [21]:
df_de

,text,lang
7,Klettern für #HambiBleibt = Kindheitserinneru...,de
8,Am 8.12. ist wieder offenes Hambi-Vernetzungst...,de
10,Und das ist auch gut so! #hambibleibt #Hambach...,de
13,"Haftprüfungen morgen Freitag, 23.2., ab 12 Uhr...",de
14,Bei dem Säureanschlag auf den Manager des RWE-...,de
15,Aktuell sind 4 Menschen in der Gesa. Haftricht...,de
17,"Es gibt keinen Grund, den #HambacherWald schon...",de
18,#HambiBleibt Aufruf zur Verteidigung der Bese...,de
19,Pofalla erklärt Einsetzung der #Kohlekommissio...,de
20,#hambisolifoto Solifototag für UP3 am kommende...,de


In [27]:
df_de.to_csv('df_de.csv')

---

In [22]:
import spacy
from spacy_sentiws import spaCySentiWS

In [23]:
# spaCySentiWS example
nlp = spacy.load('de')
sentiws = spaCySentiWS(sentiws_path='../SentiWS')
nlp.add_pipe(sentiws)

#doc = nlp('Die Dummheit der Unterwerfung blüht in hübschen Farben.')
#for token in doc:
#    print('{}, {}, {}'.format(token.text, token._.sentiws, token.pos_))

In [26]:
df_de.iloc[0]['text']

'Klettern für #HambiBleibt =  Kindheitserinnerungen und politisches Engagement verbinden 💚 https://t.co/kCfHsL9Z1d'

In [29]:
doc = nlp(df_de.iloc[0]['text'])
for token in doc: 
    print('{}, {}, {}'.format(token.text, token._.sentiws, token.pos_))

Klettern, None, NOUN
für, None, ADP
#, None, NOUN
HambiBleibt, None, PROPN
=, None, X
 , None, SPACE
Kindheitserinnerungen, None, PROPN
und, None, CONJ
politisches, None, ADJ
Engagement, 0.2157, NOUN
verbinden, 0.004, VERB
💚, None, PROPN
https://t.co/kCfHsL9Z1d, None, PROPN


In [30]:
def get_senti_score(tweet):
    """gets the score of a tweet from SentiWS"""
    scores = []
    for token in tweet:
        score = token._.sentiws
        if score is not None:
            scores.append(score)
    return sum(scores)

In [31]:
get_senti_score(doc)

0.2197

In [32]:
tweet_scores = []
for i in df_de.index:
    entry = df_de.loc[i]
    doc = nlp(entry['text'])
    tweet_score = get_senti_score(doc)
    tweet_scores.append(tweet_score)

In [33]:
df_de['score'] = tweet_scores
df_de

/home/leey/tmv/BasicBrowser/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,text,lang,score
7,Klettern für #HambiBleibt = Kindheitserinneru...,de,0.2197
8,Am 8.12. ist wieder offenes Hambi-Vernetzungst...,de,0.5983
10,Und das ist auch gut so! #hambibleibt #Hambach...,de,0.3716
13,"Haftprüfungen morgen Freitag, 23.2., ab 12 Uhr...",de,0.0040
14,Bei dem Säureanschlag auf den Manager des RWE-...,de,0.0000
15,Aktuell sind 4 Menschen in der Gesa. Haftricht...,de,0.0120
17,"Es gibt keinen Grund, den #HambacherWald schon...",de,-1.2178
18,#HambiBleibt Aufruf zur Verteidigung der Bese...,de,0.0040
19,Pofalla erklärt Einsetzung der #Kohlekommissio...,de,-0.0563
20,#hambisolifoto Solifototag für UP3 am kommende...,de,0.4836


In [34]:
df_de.to_csv("df_de_scores.csv")

In [46]:
most_neg = df_de.sort_values(by=['score'])[0:5]
for i in most_neg.index:
    print(most_neg.loc[i]['text'])

Was denn für ne Eskalation? Da konnte man schlichtweg Dummheit bei der Arbeit zusehen. Nix weiter. Und Dummheit ist gefährlich, dadurch wurde der Wald zum „gefährlichen Ort“. #HambiBleibt und #Dummheit geht, gell @hreul @CDUNRW_de https://t.co/1DBZtbmw3u
Wer nach Kennzahlen Unternehmen lenkt, wird eben mit schlechten Kennzahlen bestraft. Einfach euren Stromvertrag bei RWE und Innogy kündigen. Heute gemacht. #feelsgood #NoMoreRWE #NoMoreInnogy #HambacherForst #hambibleibt https://t.co/FgS7tmEy4O
An die Polizisten, die solche Befehle nicht ablehnen, RWE helfen den Wald zu vernichten und Gewalt an Aktivistis pr… https://t.co/ClmntHu68t
Es gibt keinen Grund, den #HambacherWald schon ab diesem Herbst zu roden. Das zeigt die Antwort der Landesregierung auf eine Kleine Anfrage von @wibkegt. Sie fordert: Keine unnötige, irreparable Rodung, Ergebnisse der #Kohlekommission abwarten! https://t.co/Ejtw2l4sym https://t.co/nxlKK0ITQv
Das gilt nicht nur für Ökologie und Frieden, sondern auch für die 

In [47]:
most_pos = df_de.sort_values(by=['score'], ascending=False)[0:5]
for i in most_pos.index:
    print(most_pos.loc[i]['text'])

Nachdem sich die #Politik in den letzten Wochen so dermaßen #professionell zum #Klimawandel geäußert hat kann ich nur hoffen, dass #EndeGelaende ein voller Erfolg wird. #hambibleibt #alledoerferbleiben #endcoal #FridaysForFuture #ClimateStrike  https://t.co/jG2MrqUTlm
Nicht vergessen. Am Sa. 6.10. ist die große Wald schützen - Kohle stoppen Demo direkt am #HambacherForst. Treffpunkt 12 Uhr am Bhf. Buir. Also packt warme Klamotten ein und malt noch schnell ein Banner. Wir sehen uns im Wald #HambiBleibt
Foto: Anjana Perera https://t.co/IUgqdvQO0Y
Laut @ZDF-Politbarometer ist 73% der Deutschen ein schnellstmöglicher Kohleausstieg wichtig oder sehr wichtig. Das wünschen wir uns auch von der #Kohlekommission! https://t.co/xGszdRfkNz https://t.co/W0LTj8e7rR
Merkel für mehr #Pillepalle beim #Klimaschutz
Na gut, dann nehmen wir den #Kohleausstieg eben weiter selbst in die Hand. Wichtige Dinge sollte Mensch ja eh keiner Regierung überlassen.
#hambibleibt #EndeGelaende
#FridaysForFuture https://

---

In [9]:
texts = tm.Status.objects.filter(
    id__in=ttweets
).values_list('text',flat=True)

In [10]:
texts[0]

'Save the hambach forest from being felled for profit! #myfirstTweet #HambacherForst #HambiBleibt!'